# Wstęp do uczenia maszynowego - Projekt 1
## Aleksander Malinowski | Damian Skowroński
___
### Milestone 2
Będziemy robić modele na danych, na których wykonaliśmy preprocessing w kamieniu milowym 1. W preprocessing wykonaliśmy następujące zmiany:

*   dodatnie kolumn:
    *   _black_king_dst_ - odleglość czarnego króla od brzegu planszy
    *   _black2white_king_dst_ - odległość króli od siebie 
    *   _black2rook_dst_ - odległość czarnego króla od białej wieży
*   wykonanie label encoding dla zmiennych określających _file_
*   zamienienie wartości słownych w zmiennej celu _result_ na odpowiadające im liczby i wartości "draw" na "-1"

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

df = pd.read_csv("training_set.csv", index_col=0).reset_index(drop=True)
df.head()

,white_king_file,white_king_rank,white_rook_file,white_rook_rank,black_king_file,black_king_rank,result,black_king_dst,black2white_king_dst,black2rook_dst
0,2,2,3,3,8,3,13,0,6,5
1,1,1,6,3,5,3,-1,2,4,1
2,4,4,7,5,8,2,6,0,4,3
3,3,1,7,3,2,7,13,1,6,5
4,4,1,2,5,8,5,13,0,4,6


Podział na zbiór treningowy i walidacyjny

Podchodzimy do problemu przewidzenia rezultatu w następujący sposób:

1.  klasyfikacja, czy doszło do remisu "draw" (u nas wartość "-1"), czy nie (dowolna inna wartość)
2. regresja w celu przewidzenia ilości ruchów, dla wyników, które w pierwszym kroku zostały sklasyfikowane, że gra nie skończyła się remisem



### Klasyfikacja 
Ponieważ zbiór danych teoretycznie przedstawia optymalne ruchy, a przewaga białego nad czarnym jest duża, remis zachodzi zapewne w chwili kiedy czarny bije białą wieżę. Taka sytuacja zdaży się w momentcie, gdy czarny król jest w odległości od białej wieży równej "1", a biały król jest w odległości od białej wieży większej niż "1". W tym celu tworzę nową zmienną _white2rook_dst_, której nie braliśmy pod uwagę w preprocessingu (możliwe, że przyda się też potem w regresji).

In [3]:
def white2rookdistance(df):
     # dystans pomiędzy białym królem i wieżą
    return max(abs(df['white_king_file'] - df['white_rook_file']),abs(df['white_king_rank'] - df['white_rook_rank']))

Następnie ze otrzymanej nowej zmiennej _white2rook_dst_ i wcześniejszej _black2rook_dst_ mogę dowiedzieć się czy czarny zbije wieżę.

In [4]:
def w2r_rt(df): #white to rook distance + rook taken
    df['white2rook_dst'] = df.apply(lambda x: white2rookdistance(x), axis = 1)
    df["rook_taken"] = np.where(
        (df.black2rook_dst == 1) & (df.white2rook_dst != 1),
        0,1)

Sprawdzając korelację pomiędzy nową zmienna _rook_taken_, a result w wersji remis/nieremis otrzymujemy następujący wynik:

In [5]:
w2r_rt(df)
np.corrcoef(df.rook_taken,df.result.apply(lambda x: 0 if x == -1 else 1))

array([[1.        , 0.99858054],
       [0.99858054, 1.        ]])

Wychodzi, że korelacja jest bardzo duża.

In [6]:
from sklearn.metrics import confusion_matrix
confusion_matrix(df.rook_taken,df.result.apply(lambda x: 0 if x == -1 else 1))

array([[ 1952,     0],
       [    5, 17682]], dtype=int64)

Okazuje się, nowa dokładnie opisuję sytuacje i popełniła tylko 5 błędów. Wobec tego myślę, że nie trzeba robić modelu do sklasyfikowania, a po prostu zdać się na nią. 

### 2. Regresja
Regresję wykonujemy w celu dowiedzenia się ile ruchów będzie potrzebne do zamatowania. Wykonywać ją będziemy na części obserwacji, które w poprzedniej "klasyfikacji" nie zwróciły remisu, czyli po prostu na tych, które mają "0" w nowej zmiennej _rook_taken_. 


In [7]:
def drop_predicted_draws(df):
    # do regresji zostawiamy tylko te wiersze w których nie ma remisu
    # na początku sortujemy w ten spoosb, aby remisy były na początku, żeby potem łatwiej połączyć ramkę, i nie musiec pamietac o indexach
    df_sorted = df.sort_values("rook_taken").reset_index(drop=True)
    df_draws = df_sorted.drop(index=np.where(df_sorted.rook_taken != 0)[0])
    df_sorted_dropped = df_sorted.drop(index=np.where(df_sorted.rook_taken == 0)[0])
    df_sorted_dropped.value_counts('result')
    return df_draws,df_sorted_dropped

df_draws, df_sorted_dropped = drop_predicted_draws(df)

Dalej podzielimy zbiór przeznaczony dla regresji.

In [8]:
train_X, test_X, train_y, test_y = train_test_split(df_sorted_dropped.drop(columns = ["result"]), df_sorted_dropped.result, 
                                                    test_size = 0.3, random_state= 420, stratify=df_sorted_dropped.result)

I poszukamy modelu, który będzie dobrze przewidywał:


#### 1. SVM

In [9]:
from sklearn import svm
regr = svm.SVR()
regr.fit(train_X,train_y)
regr.score(test_X,test_y)

0.8042820769243767

In [10]:
svr_rbf = svm.SVR(kernel="rbf", C=10, gamma=0.1, epsilon=0.1)
svr_lin = svm.SVR(kernel="linear", C=10, gamma="auto")
svr_poly = svm.SVR(kernel="poly", C=10, gamma="auto", degree=2, epsilon=0.1, coef0=1)


In [11]:
svr_rbf.fit(train_X,train_y)
svr_rbf.score(test_X,test_y)

0.8861754774551369

In [12]:
svr_lin.fit(train_X,train_y)
svr_lin.score(test_X,test_y)

0.6083084061232793

In [13]:
svr_poly.fit(train_X,train_y)
svr_poly.score(test_X,test_y)

0.7694078190101433

Najlepiej spisał się model z kernelem "rbf".

#### 2. Regresja liniowa 

In [14]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_X,train_y)
lr.score(test_X,test_y)

0.6267596907146789

#### 3. Lasso

In [15]:
from sklearn.linear_model import Lasso
las = Lasso(alpha=0.1)
las.fit(train_X,train_y)
las.score(test_X,test_y)

0.6236971549427213

#### 4. Kernel Ridge

In [16]:
from sklearn.kernel_ridge import KernelRidge
krr = KernelRidge(alpha=0.1)
krr.fit(train_X,train_y)
krr.score(test_X,test_y)

0.6267532936351177

#### Wnioski
Wygląda na to, że zdecydowanie najlepiej spisuje się SVM z kernelem "rbf". Wobec tego zajmiemy się znalezieniem odpowiednich hiperparametrów używająć opytmalizacji bayesowskiej. (random search i grid search nie sprawdzaly sie dobrze)

In [17]:
# from skopt import BayesSearchCV
# from skopt.space import Integer, Real

# #przykladowe
# model = svm.SVR(kernel="rbf")
# bayes = BayesSearchCV(model,search_spaces={
#     "C" : Integer(20,100),
#     "degree" : Integer(1,10),
#     "gamma" : Real(0,1),
#     "epsilon": Real(0,1)
# },n_jobs=-1,cv = 5)

# bayes.fit(train_X,train_y)

Użyliśmy optymalizacji bayesowskiej kilka razy na różne sposoby i otrzymaliśmy różne rezultaty jeśli chodzi o hiperparametry, lecz podobne wyniki score. Wobec tego wydaje się, że hiperparametry, oprócz "kernel", w tym przypadku nie mają dużego znaczenia.

In [18]:
bayes1 = svm.SVR(kernel = 'rbf',C=41, degree=2, epsilon=0.5453896028447068, gamma=0.29095860195816414)
bayes2 = svm.SVR(kernel = 'rbf',C=30, degree=5, epsilon=0.1967759806539503, gamma=0.08869823764675565)
bayes3 = svm.SVR(kernel = 'rbf',C=20, degree=1, epsilon=0.2010835792844748, gamma=0.09924388919619698)

In [19]:
bayes1.fit(train_X,train_y)
bayes2.fit(train_X,train_y)
bayes3.fit(train_X,train_y)

SVR(C=20, degree=1, epsilon=0.2010835792844748, gamma=0.09924388919619698)

Osiągają one na części testowej następujące wyniki: 

In [20]:
print("bayes1:",bayes1.score(test_X,test_y),
      "\nbayes2:",bayes2.score(test_X,test_y),
      "\nbayes3:",bayes3.score(test_X,test_y),
      "\nsvr_rbf (sprawdzany wcześniej):", svr_rbf.score(test_X,test_y))

bayes1: 0.8741907775193402 
bayes2: 0.8890238526140528 
bayes3: 0.8888379588129027 
svr_rbf (sprawdzany wcześniej): 0.8861754774551369


Widać, że trzy ostatnie sobie dobrze radzą i są między nimi nieznaczne różnice.

In [21]:
from sklearn.ensemble import VotingRegressor

vr = VotingRegressor(estimators=[('b2', bayes2), ('b3', bayes3), ('def', svr_rbf)],
                       n_jobs = -1)
vr.fit(train_X,train_y)
vr.score(test_X,test_y)

0.8888849796205536

In [22]:
from sklearn.ensemble import BaggingRegressor

br = BaggingRegressor(base_estimator=bayes2,n_jobs = -1)
br.fit(train_X,train_y)
br.score(test_X,test_y)

0.8912347334643809

Ensablingi też dużo nie zmieniły, chociaż BaggingRegressor pozwolił otrzymać wynik $0.89$. Wobec tego ostatecznie będzie wykorzystywany.

## Wyniki

Zgarniając naszą ramkę danych w całość (to znaczy z powrotem łącząc remisy z resztą), otrzymamy ostateczny wynik score naszego modelu.

In [23]:
from sklearn.metrics import r2_score

def get_predictions_fulldf(df_draws,df_sorted_dropped,estimator):
    # 
    regr_pred = estimator.predict(df_sorted_dropped.drop(columns="result"))
    draws = np.ones(df_draws.shape[0])*-1 #-1 oznaczało u nas wcześniej draw
    prediction = np.concatenate((draws,regr_pred))
    df_concat = pd.concat([df_draws,df_sorted_dropped])
    return df_concat,prediction

df_concat,prediction = get_predictions_fulldf(df_draws,df_sorted_dropped,br)

In [24]:
r2_score(df_concat.result,prediction)

0.9739783658354186

In [27]:
# TEST NA KLASYFIKATORACH I NA SZCZESCIE SA GORSZE xd

In [26]:
from sklearn import svm

test = svm.SVC()
test.fit(train_X,train_y)
test.score(test_X,test_y)

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(train_X,train_y)
rfc.score(test_X,test_y)

0.7015262860373093